In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
#GPU
cuda = True
use_cuda = cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print('Using ',device)

Using  cuda


In [3]:
class ClassifyNet(nn.Module):
    def __init__(self, num_in, num_hidden, num_out):
        super(ClassifyNet, self).__init__()
        self.fc1 = nn.Linear(num_in,num_hidden)
        self.fc2 = nn.Linear(num_hidden,num_out)

    def forward(self, x):
        return F.log_softmax(self.fc2(F.relu(self.fc1(x))), dim=1)

In [4]:
#Load Data from CSV and train test split
dataset = pd.read_csv('CleanedData _pytorch.csv')
dataset = np.array(dataset)

data = dataset[:, :-1]
target = dataset[:, -1]

random_seed = 43
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=0.2,random_state=random_seed)

#Convert data to torch and device
x_train_tensor = torch.from_numpy(x_train).float().to(device)
x_test_tensor = torch.from_numpy(x_test).float().to(device)
y_train_tensor = torch.from_numpy(y_train).long().to(device)
y_test_tensor = torch.from_numpy(y_test).long().to(device)

In [5]:
num_hidden = 128

#Setup model and optimizer
model = ClassifyNet(42,num_hidden,3).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.005)  #Learning rates

In [6]:
print('Iteration,Testing Accuracy,Training Accuracy')
for i in range(20):

    #Model evaluation
    model.eval()
    with torch.no_grad():
        pred = torch.argmax(model(x_test_tensor),axis=1)
        test_accuracy = torch.sum(pred == y_test_tensor)/len(pred)
        pred = torch.argmax(model(x_train_tensor),axis=1)
        train_accuracy = torch.sum(pred == y_train_tensor)/len(pred)
        print(i,test_accuracy.item()*100,train_accuracy.item()*100)

    #Training mode, run data through neural network in mini-batches
    model.train()
    for j in range(1000):
        optimizer.zero_grad()
        loss = F.nll_loss(model(x_train_tensor), y_train_tensor)
        loss.backward()
        optimizer.step()

Iteration,Testing Accuracy,Training Accuracy
0 10.546962171792984 11.052106320858002
1 82.08127021789551 85.18059849739075
2 82.11087584495544 86.314857006073
3 81.86662793159485 86.78484559059143
4 82.13307857513428 87.060546875
5 82.11827874183655 87.19192147254944
6 82.20709562301636 87.20487356185913
7 82.21449255943298 87.42506504058838
8 82.18488693237305 87.2566819190979
9 82.28110671043396 87.42876648902893
10 81.97025060653687 87.61379718780518
11 82.15528130531311 87.54533529281616
12 82.14788436889648 87.46762275695801
13 81.88883662223816 87.62490153312683
14 82.06647038459778 87.61935234069824
15 81.97025060653687 87.6230537891388
16 81.94804787635803 87.78218030929565
17 82.16268420219421 87.59159445762634
18 81.97025060653687 87.70816922187805
19 81.82222247123718 87.76922821998596


In [16]:
torch.argmax(model(x_test_tensor[:20]),axis=1)

tensor([1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0],
       device='cuda:0')

In [17]:
y_test_tensor[:20]

tensor([1, 1, 1, 0, 0, 1, 2, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       device='cuda:0')